In [10]:
import os
import random
from collections import defaultdict
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# ===== LLM =====
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=os.getenv("OPENAI_API_KEY")
)

# ===== TOOL TRACKER =====
class ToolTracker:
    def __init__(self):
        self.calls = []
        self.counts = defaultdict(int)

    def track(self, tool_name):
        self.calls.append(tool_name)
        self.counts[tool_name] += 1

    def stats(self):
        return {
            "total_calls": len(self.calls),
            "counts": dict(self.counts),
            "sequence": self.calls,
            "most_used": max(self.counts, key=self.counts.get) if self.counts else None
        }

tracker = ToolTracker()

# ===== TOOLS =====
@tool
def consult_demogorgon(complaint: str) -> str:
    """Get the Demogorgon's perspective on a complaint about the Upside Down."""
    tracker.track("consult_demogorgon")
    responses = [
        f"The Demogorgon tilts its head. It seems confused by '{complaint}'. Perhaps the issue is that you're thinking in three dimensions?",
        f"The Demogorgon growls mysteriously about '{complaint}'. It hints that time works differently in the Upside Down.",
        f"The Demogorgon is eating something. It ignores '{complaint}', showing that chaos governs its choices."
    ]
    return random.choice(responses)

@tool
def check_hawkins_records(query: str) -> str:
    """Search Hawkins historical records for patterns or explanations."""
    tracker.track("check_hawkins_records")
    records = {
        "portal": "Portals have opened on various dates with no clear pattern; energy surges and location may influence them.",
        "monsters": "Creatures act differently depending on environmental factors, time of day, and proximity to psychics.",
        "psychics": "Abilities vary greatly; some individuals can move objects, others see visions but cannot move things.",
        "electricity": "Electrical anomalies in Hawkins suggest a connection to the Upside Down's electromagnetic fields."
    }
    for key, value in records.items():
        if key in query.lower():
            return value
    return f"No direct records for '{query}', but strange events are common in Hawkins."

@tool
def cast_interdimensional_spell(problem: str, creativity_level: str="medium") -> str:
    """Suggest a creative interdimensional spell to solve a problem."""
    tracker.track("cast_interdimensional_spell")
    multiplier = {"low": 1, "medium": 2, "high": 3}[creativity_level]
    spells = [
        f"Chant 'Bemca Becma Becma' three times with a Walkman to stabilize: {problem}",
        f"Create a salt circle and place a compass inside. Magnetic anomalies might stabilize: {problem}",
        f"Play 'Running Up That Hill' backwards at the location. Temporal resonance may fix: {problem}",
        f"Arrange three items (lighter, compass, personal item) in a triangle while focusing on: {problem}"
    ]
    selected = random.sample(spells, min(multiplier, len(spells)))
    return "\n".join(selected)

@tool
def gather_party_wisdom(question: str) -> str:
    """Ask the D&D party for their collective wisdom."""
    tracker.track("gather_party_wisdom")
    party_responses = {
        "portal": "Mike: 'Portals are unpredictable but linked to strong emotional events.' Dustin: 'They may follow Mind Flayer activity patterns.'",
        "monsters": "Lucas: 'Demogorgons are opportunistic.' Will: 'They sense fear and act differently depending on emotion.'",
        "psychics": "Mike: 'El's powers vary with her emotional state.' Dustin: 'Energy limits her actions.'",
        "electricity": "Lucas: 'The Upside Down interferes with electrical systems.' Dustin: 'It can create feedback loops.'"
    }
    for key, response in party_responses.items():
        if key in question.lower():
            return response
    return "The party is unsure and suggests further investigation."

tools = [
    consult_demogorgon,
    check_hawkins_records,
    cast_interdimensional_spell,
    gather_party_wisdom
]

print(f"Created {len(tools)} creative tools:")
for t in tools:
    print(f"  - {t.name}: {t.description[:60]}...")

# ===== CREATE AGENT =====
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a paranormal investigator in Hawkins."
)

# ===== TEST COMPLAINTS =====
complaints = [
    "Why do demogorgons sometimes eat people and sometimes don't?",
    "Is there a schedule for portal openings?",
    "Why do psychic powers vary between people?",
    "Why does electricity behave strangely near creatures?"
]

for complaint in complaints:
    print("\n" + "="*60)
    print("COMPLAINT:", complaint)
    print("="*60)

    result = agent.invoke({
        "messages": [("user", complaint)]
    })

    print("\nFINAL RESPONSE:")
    print(result["messages"][-1].content)

# ===== TOOL USAGE ANALYSIS =====
print("\n" + "="*60)
print("TOOL USAGE ANALYSIS")
print("="*60)

stats = tracker.stats()
print("Total tool calls:", stats["total_calls"])
print("Tool usage counts:", stats["counts"])
print("Most used tool:", stats["most_used"])
print("Call sequence:", " -> ".join(stats["sequence"]))


Created 4 creative tools:
  - consult_demogorgon: Get the Demogorgon's perspective on a complaint about the Up...
  - check_hawkins_records: Search Hawkins historical records for patterns or explanatio...
  - cast_interdimensional_spell: Suggest a creative interdimensional spell to solve a problem...
  - gather_party_wisdom: Ask the D&D party for their collective wisdom....

COMPLAINT: Why do demogorgons sometimes eat people and sometimes don't?

FINAL RESPONSE:
The Demogorgon seems puzzled by the question, indicating that understanding its behavior may require thinking beyond our three-dimensional perspectives. 

The D&D party is also unsure, suggesting that further investigation might be necessary to uncover theories about its eating habits. It might be worth exploring the Demogorgon's motivations, environmental factors, or even its hunger cycles to gain more insight. Would you like to investigate any specific aspect further?

COMPLAINT: Is there a schedule for portal openings?

FINA